## Testing of the Graph Environment

Imports

In [1]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym

import ray


/Users/noah/opt/anaconda3/lib/python3.8/site-packages/redis/connection.py:72: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)


In [2]:
from ray.rllib.agents.dqn import DQNTrainer, DEFAULT_CONFIG

Import the Environment

In [3]:
sys.path.insert(0,"")

from ManhattanGraph import ManhattanGraph
from gym_graphenv.envs.GraphworldManhattan import GraphEnv

In [4]:
env=GraphEnv()

/Users/noah/opt/anaconda3/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
/Users/noah/opt/anaconda3/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
/Users/noah/opt/anaconda3/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
/Users/noah/opt/anaconda3/lib/python3.8/site-packages/osmnx/distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.n

Run random steps without Ray Trainer

In [5]:
def run_one_episode (env):
    env.reset()
    sum_reward = 0
    for i in range(30):
        #print(env.action_space)
        env.available_actions = env.get_available_actions()
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print("DELIVERY DONE! sum_reward: ",sum_reward, " time: ",env.time,  "deadline time: ", env.deadline,"pickup time: ", env.pickup_time )
            break

        print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

for i in range(10):
    run_one_episode (env)

get_available_actions() Execution time: 1.650068998336792 seconds
action ==  ownRide 
Compute_reward() Execution time: 1.9073486328125e-06 seconds
Step() Execution time: 0.001277923583984375 seconds
DELIVERY DONE! sum_reward:  100  time:  2016-01-01 06:14:31 deadline time:  2016-01-01 09:08:00 pickup time:  2016-01-01 06:08:00
get_available_actions() Execution time: 1.4263889789581299 seconds
action == wait 
Compute_reward() Execution time: 5.7220458984375e-06 seconds
Step() Execution time: 0.00028586387634277344 seconds
sum_reward:  -0.027777777777777776  time:  2016-01-01 06:13:00 deadline time:  2016-01-01 09:08:00 pickup time:  2016-01-01 06:08:00
get_available_actions() Execution time: 2.0387980937957764 seconds
action == wait 
Compute_reward() Execution time: 9.059906005859375e-06 seconds
Step() Execution time: 0.001283884048461914 seconds
sum_reward:  -0.05555555555555555  time:  2016-01-01 06:18:00 deadline time:  2016-01-01 09:08:00 pickup time:  2016-01-01 06:08:00
get_availa

Initialize Ray

In [7]:
ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': 'tcp://127.0.0.1:64621',
 'raylet_socket_name': 'tcp://127.0.0.1:62671',
 'webui_url': None,
 'session_dir': 'C:\\Users\\cosmi\\AppData\\Local\\Temp\\ray\\session_2022-04-15_19-34-34_080168_14936',
 'metrics_export_port': 56577,
 'gcs_address': '127.0.0.1:64165',
 'address': '127.0.0.1:64165',
 'node_id': '3c53aa6b2ece291ebebd785433eeba23168b4510525c79f85632883e'}

Set trainer configuration

In [8]:
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['num_workers'] = 1
trainer_config["train_batch_size"] = 400
trainer_config["gamma"] = 0.95
trainer_config["n_step"] = 10
trainer_config["framework"] = "torch"
#trainer_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0"))


Initialize the trainer

In [9]:
trainer = DQNTrainer(trainer_config,GraphEnv )

2022-04-15 19:34:45,669	INFO simple_q.py:154 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
2022-04-15 19:34:45,670	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2022-04-15 19:36:05,189	INFO trainable.py:127 -- Trainable.setup took 79.524 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-15 19:36:05,192	WARNING util.py:55 -- Install gputil for GPU system monitoring.


Define the path where the results of the trainer should be saved

In [10]:
checkpoint_root = "tmp/dqn/graphworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

Run the trainer

In [11]:
results = []
episode_data = []
episode_json = []
n_iter=10

for n in range(n_iter):
    result = trainer.train()
    results.append(result)
    print("TEST",n)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

 pid=9980) action ==  ownRide 
 pid=9980) action ==  ownRide 
 pid=9980) action ==  ownRide 
 pid=9980) action ==  ownRide 
 pid=9980) action ==  ownRide 
 pid=9980) action ==  ownRide 
 pid=9980) action ==  ownRide 


2022-04-15 19:38:52,771	WARNING trainer.py:975 -- Worker crashed during call to `step_attempt()`. To try to continue training without the failed worker, set `ignore_worker_failures=True`.


 pid=9980) action == wait 
 pid=9980) Nonerker


RayTaskError(TypeError): [36mray::RolloutWorker.par_iter_next()[39m (pid=9980, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x000001938F580A90>)
  File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 644, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 593, in ray._raylet.execute_task.function_executor
  File "C:\ProgramData\Anaconda3\envs\teamproject\lib\site-packages\ray\_private\function_manager.py", line 648, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\teamproject\lib\site-packages\ray\util\tracing\tracing_helper.py", line 451, in _resume_span
    return method(self, *_args, **_kwargs)
  File "C:\ProgramData\Anaconda3\envs\teamproject\lib\site-packages\ray\util\iter.py", line 1151, in par_iter_next
    return next(self.local_it)
  File "C:\ProgramData\Anaconda3\envs\teamproject\lib\site-packages\ray\rllib\eva

In [ ]:
results

Testing of the trainer

In [ ]:
trainer.restore(file_name)
env = gym.make("graphworld-v0")
state = env.reset()

In [ ]:
sum_reward = 0
n_step = 20
for step in range(n_step):
    action = trainer.compute_action(state)
    state, reward, done, info = env.step(action)
    sum_reward += reward
    #env.render()
    if done == 1:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

In [ ]:
ray.shutdown()